In [5]:
# Data Harvesters, v1.7
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
import requests
from datetime import datetime, timedelta
import traceback
import numpy as np

# ==============================================================================
# --- БЛОК 1: КОНФИГУРАЦИЯ И ПОДКЛЮЧЕНИЕ ---
# ==============================================================================
CREDS_FILE = 'credentials.json'
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1qBYS_DhGNsTo-Dnph3g_H27aHQOoY0EOcmCIKarb7Zc/"
SCOPE = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive.file']

def get_gsheets_client(creds_file, scope):
    try:
        creds = Credentials.from_service_account_file(creds_file, scopes=scope)
        client = gspread.authorize(creds)
        print("✅ Авторизация в Google Sheets прошла успешно.")
        return client
    except Exception as e:
        print(f"❌ Ошибка авторизации Google: {e}"); return None

# ==============================================================================
# --- БЛОК 2: ФУНКЦИИ-СБОРЩИКИ ---
# ==============================================================================
def get_moex_history(ticker: str, start_date: str, market: str, board: str) -> pd.DataFrame:
    print(f"  - Запрос истории для {ticker} ({market}/{board}) с даты {start_date}...")
    url = f"https://iss.moex.com/iss/history/engines/stock/markets/{market}/boards/{board}/securities/{ticker}.json?from={start_date}&iss.meta=off"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json().get('history', {})
        if not data.get('data'):
            print(f"    - ⚠️ Для {ticker} не вернулась история.")
            return pd.DataFrame()

        cols = data['columns']
        df = pd.DataFrame(data['data'], columns=cols)
        # Выбираем только нужные столбцы
        required_cols = ['TRADEDATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME']
        df = df[required_cols]
        df.rename(columns={'TRADEDATE': 'Date'}, inplace=True)
        return df
    except Exception as e:
        print(f"    - ❌ Ошибка при получении истории для {ticker}: {e}")
        return pd.DataFrame()

# ==============================================================================
# --- БЛОК 3: ГЛАВНАЯ ЛОГИКА ---
# ==============================================================================
def main_history_updater():
    print("\n" + "="*50)
    print("--- ✨ АСУП ИИ: Обновление Истории v1.8 (с санитизацией) ✨ ---")
    print("="*50)

    client = get_gsheets_client(CREDS_FILE, SCOPE)
    if not client: return

    try:
        spreadsheet = client.open_by_url(SPREADSHEET_URL)
        holdings_sheet = spreadsheet.worksheet('Holdings')
        history_sheet = spreadsheet.worksheet('History_OHLCV')
    except Exception as e:
        print(f"❌ КРИТИЧЕСКАЯ ОШИБКА: Не могу открыть таблицу или листы. {e}"); return

    holdings_df = pd.DataFrame(holdings_sheet.get_all_records())
    history_df = pd.DataFrame(history_sheet.get_all_records() if history_sheet.row_count > 1 else [])

    tickers_to_process = holdings_df[holdings_df['Type'].isin(['Stock_MOEX', 'Bond_MOEX'])]['Ticker'].tolist()

    new_history_rows = []

    for ticker in tickers_to_process:
        asset_type = holdings_df[holdings_df['Ticker'] == ticker]['Type'].iloc[0]
        market, board = ('shares', 'TQBR') if asset_type == 'Stock_MOEX' else ('bonds', 'TQOB')

        last_date_str = None
        if not history_df.empty and 'Ticker' in history_df.columns:
            last_date_str = history_df[history_df['Ticker'] == ticker]['Date'].max()

        start_date = (datetime.strptime(last_date_str, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d') if pd.notna(last_date_str) else (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

        ticker_history_df = get_moex_history(ticker, start_date, market, board)

        if not ticker_history_df.empty:
            # ИСПРАВЛЕНИЕ: "Санитизация" данных перед записью
            # Заменяем бесконечные значения на NaN (Not a Number)
            ticker_history_df.replace([np.inf, -np.inf], np.nan, inplace=True)
            # Заменяем NaN на пустую строку, чтобы gspread записал пустую ячейку
            ticker_history_df.fillna('', inplace=True)

            for _, row in ticker_history_df.iterrows():
                new_history_rows.append([row['Date'], 'D1', ticker, row['OPEN'], row['HIGH'], row['LOW'], row['CLOSE'], row['VOLUME']])

    if new_history_rows:
        print(f"\n🔄 Найдено {len(new_history_rows)} новых записей. Добавляю в 'History_OHLCV'...")
        history_sheet.append_rows(new_history_rows, value_input_option='USER_ENTERED')
        print(f"✅ История успешно дополнена.")
    else:
        print("✅ Новых исторических данных для акций и облигаций не найдено.")

    print("--- 🏁 РАБОТА ОБНОВИТЕЛЯ ИСТОРИИ ЗАВЕРШЕНА 🏁 ---")

if __name__ == "__main__":
    main_history_updater()


--- ✨ АСУП ИИ: Обновление Истории v1.8 (с санитизацией) ✨ ---
✅ Авторизация в Google Sheets прошла успешно.
  - Запрос истории для GAZP (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для ROSN (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для SIBN (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для NVTK (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для SNGS (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для SNGSP (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для TATN (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для TRNFP (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для AFLT (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для GLTR (shares/TQBR) с даты 2024-06-09...


<ipython-input-5-18e9d920b1d0>:92: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ticker_history_df.fillna('', inplace=True)


  - Запрос истории для SBER (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для SBERP (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для TCSG (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для MOEX (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для AFKS (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для YNDX (shares/TQBR) с даты 2024-06-09...


<ipython-input-5-18e9d920b1d0>:92: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ticker_history_df.fillna('', inplace=True)


  - Запрос истории для MTSS (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для MGNT (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для FIVE (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для GMKN (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для PLZL (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для CHMF (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для PHOR (shares/TQBR) с даты 2024-06-09...
  - Запрос истории для ОФЗ 26238 (bonds/TQOB) с даты 2024-06-09...
    - ⚠️ Для ОФЗ 26238 не вернулась история.
  - Запрос истории для ОФЗ 26227 (bonds/TQOB) с даты 2024-06-09...
    - ⚠️ Для ОФЗ 26227 не вернулась история.
  - Запрос истории для ОФЗ 29014 (bonds/TQOB) с даты 2024-06-09...
    - ⚠️ Для ОФЗ 29014 не вернулась история.
  - Запрос истории для ОФЗ 52005 (bonds/TQOB) с даты 2024-06-09...
    - ⚠️ Для ОФЗ 52005 не вернулась история.

🔄 Найдено 2221 новых записей. Добавляю в 'History_OHLCV'...
✅ История успешно дополнена.
--- 🏁 РАБОТА ОБНОВИТЕ